Este colab fue desarrollado por Arnold Charry Armero y Rolando Acosta Amado.

# El Modelo EOQ con Restricciones

El Modelo EOQ clásico se enfoca principalmente en sólo un artículo. Es importante destacar que cuando se tiene más de un artículo en inventario y estos comparten los mismos recursos, se debe de realizar un análisis más minucioso (Sipper & Bulfin, 1998). Por ello, se resuelve este problema como uno de optimización no lineal con restricciones; esto es, utilizar el método matemático de Multiplicadores de Lagrange para minimizar los costos totales promedio. Cabe destacar que las restricciones que por lo general más aparecen en este tipo de problemas son las de espacio (almacenamiento) y presupuesto (inversión).

**Supuestos:**

*   Existen varios productos en el sistema de inventarios
*   La demanda es uniforme y determinística
*   No se permiten faltantes
*   No hay un tiempo de entrega
*   Tasa de reabastecimiento infinita, es decir, toda la cantidad ordenada llega al mismo tiempo.


Matemáticamente, el modelo se expresa de la siguiente manera:

$$K(Q_{i},\:\lambda)=\sum_{i=1}^{n}(c_{i}D_{i} + \frac{A_{i}D_{i}}{Q_{i}} + h_{i} \frac{Q_{i}}{2}) +\lambda \left \{ \sum_{i=1}^{n}f_{i}Q_{i}-L \right \}$$

donde,

$ Q = \mathrm{Cantidad \:de \: lote \: a \: ordenar} $

$ c = \mathrm{Costo \: unitario \: de \: compra} $

$ D = \mathrm{Tasa \:de \: demanda} $

$ A = \mathrm{Costo \:unitario \: de \: ordenar} $

$ h = \mathrm{Costo \:unitario \: de \: mantener \: en \: inventario} $

$ \lambda = \mathrm{Precio \:sombra \: (Restricción)} $

$ f = \mathrm{Valor \: unitario \: para \: la \: restricción} $

$ L = \mathrm{Valor \: máximo \: de \: restricción} $

$ n = \mathrm{Cantidad \: de \: productos} $

$ K = \mathrm{Costo \: total \: promedio} $

Para saber cuál es es el tamaño de lote a ordenar para cada producto, se deben realizar las $n$ derivadas parciales por producto y las $j$ derivadas parciales para cada restricción. Posteriormente, se debe resolver el sistema de ecuaciones y se obtiene la cantidad óptima para cada producto. Esto se ve reflejado en las siguientes ecuaciones,


$$\frac{\partial K(Q, \: \lambda)}{\partial Q_{i}} = 0, \: i = 1,2,\cdots ,n$$

$$\frac{\partial K(Q, \: \lambda)}{\partial \lambda _{j}} = 0, \: j = 1,2,\cdots ,n$$

## Multiplicadores de Lagrange

Para resolver estos ejercicios, es recomendable usar una herramienta computacional. Python para resolver este tipo de ejercicios utiliza Multiplicadores de Lagrange, es decir, separa la función objetivo de las restricciones. Esto se puede ver en la siguiente notación.

$$K(Q_{i})=\sum_{i=1}^{n}c_{i}D_{i} + \frac{A_{i}D_{i}}{Q_{i}} + h_{i} \frac{Q_{i}}{2}$$

$\mathrm{s. a.}$

$$ \sum_{i=1}^{n}f_{i}Q_{i} \leq L$$

Y para la solución, utilizando Multiplicadores de Lagrange, se tiene,

$$ \vec{\nabla} K =\lambda_{1} \vec{\nabla} g_{1} + \lambda_{2} \vec{\nabla} g_{2} +\cdots+\lambda_{n} \vec{\nabla} g_{n} $$

$$\frac{\partial K}{\partial Q_{i}} = \lambda\frac{\partial G}{\partial Q_{i}} \: \:\forall_{i}$$

Ahora se continua con la implementación en código resolviendo dos ejemplos. El primer ejemplo es para una restricción. El segundo ejemplo es con dos restricciones. Se resuelve el siguiente ejemplo.

**Ejemplo 6-9. Artículos múltiples, una restricción.** HiEnd, una pequeña compañía de computadoras,
compra dos tipos de lectoras de discos. Debido al bajo volumen que maneja la compañía, el
gerente limita la inversión en inventario a un máximo de $\$5000$. El precio de estas dos lectoras es de
$\$50$ y $\$80$, respectivamente, y su demanda anual es 250 y 484 unidades, respectivamente. La compañía
tiene un gasto de $\$ 50$ para procesar la orden de cualquiera de estas lectoras, y el gerente usa un
$20$% anual para las evaluaciones financieras.

In [ ]:
#Se importan las librerias
import scipy.optimize as optimize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Se define la función
def EOQ_function(c, D, A, i, Q):
  """
  Calcula el costo total promedio tomando en
  cuenta el tamaño de lote Q ingresado

  Args:
    c: Costo unitario del producto
    D: Tasa de Demanda
    A: Costo unitario de ordenar
    el producto
    i: Porcentaje de mantener en
    inventario
    Q: Tamaño de lote seleccionado.

  Returns:
    K: Costo total promedio
  """
  if Q == 0:
    return np.inf
  return c * D + (A * D) / Q + (i * c * Q) / 2

Se tiene la siguiente formulación,

$$K(Q_{i})=\sum_{i=1}^{n}c_{i}D_{i} + \frac{A_{i}D_{i}}{Q_{i}} + h_{i} \frac{Q_{i}}{2}$$

$\mathrm{s. a.}$

$$ \sum_{i=1}^{n}c_{i}Q_{i} \leq C$$

In [ ]:
#Se insertan los parámetros
i = 0.2
df = pd.DataFrame()
df['D'] = [250, 484]
df['A'] = [50, 50]
df['c'] = [50, 80]
print(df)

     D   A   c
0  250  50  50
1  484  50  80


In [ ]:
#Se define la función de costos con la sumatoria
Q = np.array([50,50])

def total_cost(Q):
  cost = 0
  for j in range(0, len(Q)):
    cost += EOQ_function(df.c[j],df.D[j],df.A[j],i,Q[j])
  return cost

In [ ]:
#Se define la restricción
def budget_constraint(Q):
  total_expenditures = 0
  for j in range(0, len(Q)):
    total_expenditures += df.c[j] * Q[j]
  return -total_expenditures + 5000

In [ ]:
#Requerimientos para la optimización

#Restricción - Inecuación
cons_1 = {'type' : 'ineq', 'fun' : budget_constraint}
const = [cons_1]

#Límites de las variables de decisión - sólo números positivos
num_bnds = []
for j in range(len(Q)):
  tupla = (0, np.inf)
  num_bnds.append(tupla)
bnds = num_bnds

In [ ]:
#Optimización con restricciones
solution = optimize.minimize(total_cost, Q, constraints=const, method='SLSQP', bounds = bnds)
solution

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 52672.20000001145
       x: [ 3.623e+01  3.986e+01]
     nit: 6
     jac: [-4.522e+00 -7.235e+00]
    nfev: 18
    njev: 6

In [ ]:
print('------SOLUCIÓN------')
print('Se deben comprar', solution.x[0], 'unidades del producto 1.')
print('Se deben comprar', solution.x[1], 'unidades del producto 2.')
print('El costo total promedio es de ${}'.format(solution.fun))

------SOLUCIÓN------
Se deben comprar 36.23171739183659 unidades del producto 1.
Se deben comprar 39.855176630101866 unidades del producto 2.
El costo total promedio es de $52672.20000001145


Ahora se resuelve este mismo ejercicio, pero con dos restricciones.

**Ejemplo 6-10. Artículos múltiples: dos restricciones.** HiEnd no tiene mucho espacio para almacenar
las lectoras de discos. Suponga que cada tipo de lectora requiere 10 y 8 unidades de espacio,
respectivamente, y cuenta con un total de 500 unidades. ¿Satisfacen los resultados previos la restricción
de espacio? Resuelva este problema como se necesite.

Se tiene la siguiente formulación,

$$K(Q_{i})=\sum_{i=1}^{n}c_{i}D_{i} + \frac{A_{i}D_{i}}{Q_{i}} + h_{i} \frac{Q_{i}}{2}$$

$\mathrm{s. a.}$

$$ \sum_{i=1}^{n}c_{i}Q_{i} \leq C$$

$$ \sum_{i=1}^{n}f_{i}Q_{i} \leq F$$

In [ ]:
#Se insertan los datos del espacio
df['f'] = [10, 8]
print(df)

     D   A   c   f
0  250  50  50  10
1  484  50  80   8


In [ ]:
#Se define la nueva restricción
def space_constraint(Q):
  total_space = 0
  for i in range(0, len(Q)):
    total_space += df.f[i] * Q[i]
  return -total_space + 500

In [ ]:
#Requerimientos para la optimización

#Restricción - Inecuación
cons_2 = {'type' : 'ineq', 'fun' : space_constraint}
const = [cons_1, cons_2]

#Límites de las variables de decisión - sólo números positivos
num_bnds = []
for j in range(len(Q)):
  tupla = (0, np.inf)
  num_bnds.append(tupla)
bnds = num_bnds

In [ ]:
#Optimización con restricciones
solution = optimize.minimize(total_cost, Q, constraints=const, method='SLSQP', bounds = bnds)
solution

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 52864.98038538138
       x: [ 2.352e+01  3.310e+01]
     nit: 4
     jac: [-1.760e+01 -1.408e+01]
    nfev: 13
    njev: 4

In [ ]:
print('------SOLUCIÓN------')
print('Se deben comprar', solution.x[0], 'unidades del producto 1.')
print('Se deben comprar', solution.x[1], 'unidades del producto 2.')
print('El costo total promedio es de ${}'.format(solution.fun))

------SOLUCIÓN------
Se deben comprar 23.5165421726808 unidades del producto 1.
Se deben comprar 33.10432228415276 unidades del producto 2.
El costo total promedio es de $52864.98038538138


Ahora, se agrega un código que sirve para cualquier restricción genérica,

In [ ]:
def generic_constraint(list, array, right_hand_side):
  left_hand_side = 0
  for i in range(0, len(array)):
    left_hand_side += list[i] * array[i]
  return -left_hand_side + right_hand_side

cons_3 = {'type' : 'ineq', 'fun' : lambda x: generic_constraint(df.s, x, 1500)}
cons_4 = {'type' : 'ineq', 'fun' : lambda x: generic_constraint(df.c, x, 3800)}

const = [cons_3, cons_4]

Dando así, finalmente, por concluido el tema de Multiplicadores de Lagrange en Python.

#### Referencias


* Chopra, S., & Meindl, P. (2016). Supply chain management: Strategy, planning, and operation, global edition (6th ed.). Pearson Education.
*   Sipper, D., & Bulfin, R. L. (1998). Planeación y control de la producción.
*   Zill, D. G. (2011). Cálculo de varias variables (4a. ed.).